# Silver Schema (skip this)

In [ ]:
CREATE OR REPLACE TABLE `finalproject-g2df12.retail.fct_invoices` AS

SELECT
  InvoiceNo,
  PARSE_DATETIME('%m/%d/%y %H:%M', InvoiceDate) AS InvoiceDate,
  StockCode,
  CAST(CustomerID AS INT64) AS CustomerID,
  Quantity,
  CAST(UnitPrice AS NUMERIC) * CAST(Quantity AS NUMERIC) AS TotalPrice
FROM `finalproject-g2df12.retail.raw_dataiku`

In [ ]:
CREATE OR REPLACE TABLE `finalproject-g2df12.retail.dim_products` AS

SELECT
  StockCode,
  Description,
  ModeUnitPrice AS UnitPrice
FROM (
  SELECT
    StockCode,
    Description,
    UnitPrice AS ModeUnitPrice,
    ROW_NUMBER() OVER (PARTITION BY StockCode ORDER BY COUNT(*) DESC) AS rn
  FROM
    `finalproject-g2df12.retail.raw_dataiku`
  --WHERE UnitPrice > 0
  GROUP BY
    StockCode, Description, UnitPrice
)
WHERE rn = 1
ORDER BY StockCode

In [ ]:
CREATE OR REPLACE TABLE `finalproject-g2df12.retail.dim_customers` AS

SELECT
  CAST(CustomerID AS INT64) AS CustomerID,
  Country,
  MIN(InvoiceDate) AS first_transaction,
  MAX(InvoiceDate) AS last_transaction,
  COUNT(InvoiceNo) AS count_order,
  SUM(UnitPrice * Quantity) AS total_purchase
FROM `finalproject-g2df12.retail.raw_dataiku`
GROUP BY CustomerID, Country

In [ ]:
CREATE OR REPLACE TABLE `finalproject-g2df12.retail.dim_datetime` AS

SELECT
  InvoiceDate,
  EXTRACT(YEAR FROM InvoiceDate) AS year,
  EXTRACT(MONTH FROM InvoiceDate) AS month,
  EXTRACT(DAY FROM InvoiceDate) AS day,
  EXTRACT(QUARTER FROM InvoiceDate) AS quarter,
  EXTRACT(DAYOFWEEK FROM InvoiceDate) AS day_of_week,
  EXTRACT(ISOWEEK FROM InvoiceDate) AS week_of_year
FROM `finalproject-g2df12.retail.raw_dataiku`

# BigQuery Connection

In [ ]:
!pip install --upgrade google-cloud-storage

In [ ]:
import os
from google.cloud import storage

# Define your GCS bucket and the key file path
bucket_name = 'g2-finalproject'
key_file_path = 'Credential/vertex.json'
local_key_path = '/tmp/service-account-file.json'

# Initialize a client
storage_client = storage.Client()

# Download the key file from GCS
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(key_file_path)
blob.download_to_filename(local_key_path)

# Set the environment variable
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = local_key_path

# Verify the environment variable is set correctly
print(f"GOOGLE_APPLICATION_CREDENTIALS is set to: {os.getenv('GOOGLE_APPLICATION_CREDENTIALS')}")

GOOGLE_APPLICATION_CREDENTIALS is set to: /tmp/service-account-file.json


In [ ]:
from google.cloud import bigquery
import pandas as pd

# Initialize the BigQuery client
client = bigquery.Client(location="asia-southeast2")

def run_query(query):
  query_job = client.query(query)  # Create an API request
  result = query_job.result() # Wait for the query to finish

  df_query = result.to_dataframe()
  return df_query

In [ ]:
query = """
SELECT *
FROM `finalproject-g2df12.retail.raw_dataiku`
LIMIT 10
"""

run_query(query)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536414,22139,None,56,2010-12-01 11:52:00,0.00,None,United Kingdom
1,536544,22077,6 RIBBONS RUSTIC CHARM,3,2010-12-01 14:32:00,3.36,None,United Kingdom
2,536544,22111,SCOTTIE DOG HOT WATER BOTTLE,2,2010-12-01 14:32:00,11.02,None,United Kingdom
3,536544,22080,RIBBON REEL POLKADOTS,1,2010-12-01 14:32:00,3.36,None,United Kingdom
4,536544,71459,HANGING JAM JAR T-LIGHT HOLDER,8,2010-12-01 14:32:00,1.69,None,United Kingdom
5,536544,21790,VINTAGE SNAP CARDS,9,2010-12-01 14:32:00,1.66,None,United Kingdom
6,536544,22037,ROBOT BIRTHDAY CARD,1,2010-12-01 14:32:00,0.85,None,United Kingdom
7,536544,22224,WHITE LOVEBIRD LANTERN,1,2010-12-01 14:32:00,5.91,None,United Kingdom
8,536544,22379,RECYCLING BAG RETROSPOT,2,2010-12-01 14:32:00,4.21,None,United Kingdom
9,536544,85064,CREAM SWEETHEART LETTER RACK,1,2010-12-01 14:32:00,11.02,None,United Kingdom


# Data Transformation in Python

## Remove Cancelled InvoiceNo

In [ ]:
query = "SELECT * FROM `finalproject-g2df12.retail.fct_invoices`"
fct_invoices = run_query(query)
print("\nNumber of initial invoices:", (fct_invoices).shape[0])

print("\nNumber of invoices with more than 6 chars:", (fct_invoices[fct_invoices['InvoiceNo'].str.len() > 6]).shape[0])

# Filter rows where length of 'InvoiceNo' is 6 or fewer characters
fct_invoices = fct_invoices[fct_invoices['InvoiceNo'].str.len() <= 6]

print("\nNumber of cleaned invoices:", (fct_invoices).shape[0])


Number of initial invoices: 541909

Number of invoices with more than 6 chars: 9291

Number of cleaned invoices: 532618


## Remove Missing CustomerID

In [ ]:
query = "SELECT * FROM `finalproject-g2df12.retail.dim_customers`"
dim_customers = run_query(query)
print("\nNumber of initial customers:", (dim_customers).shape[0])

# Drop rows with null CustomerID
dim_customers = dim_customers.dropna(subset=['CustomerID'])
print("\nNumber of cleaned customers:", (dim_customers).shape[0])


Number of initial customers: 4389

Number of cleaned customers: 4380


## Negative UnitPrice

In [ ]:
query = "SELECT * FROM `finalproject-g2df12.retail.dim_products`"
dim_products = run_query(query)

# Check for negative UnitPrice
print("\nNumber of negative UnitPrice:", (dim_products[dim_products['UnitPrice'] < 0]).shape[0])

# Show products with negative unitprice
print("\nProducts with negative UnitPrice:")
dim_products[dim_products['UnitPrice'] < 0]


Number of negative UnitPrice: 1

Products with negative UnitPrice:


,StockCode,Description,UnitPrice
4038,B,Adjust bad debt,-11062.06


In [ ]:
# Drop rows with negative UnitPrice
dim_products = dim_products[dim_products['UnitPrice'] >= 0]
print("\nNumber of cleaned products:", (dim_products).shape[0])


Number of cleaned products: 4069


# Gold Schema

In [ ]:
CREATE OR REPLACE TABLE `finalproject-g2df12.retail.report_sales` AS

SELECT
  InvoiceNo,
  InvoiceDate,
  fct.StockCode,
  Description,
  UnitPrice,
  Quantity,
  UnitPrice * Quantity AS TotalPrice
FROM `finalproject-g2df12.retail.fct_invoices` AS fct
LEFT JOIN `finalproject-g2df12.retail.dim_products` AS prod
  ON fct.StockCode = prod.StockCode

In [ ]:
CREATE OR REPLACE TABLE `finalproject-g2df12.retail.report_customers` AS

SELECT
  DISTINCT fct.CustomerID,
  Country,
  RFM_Score,
  RFM_Segment,
  Segment,
  COUNT(InvoiceNo) OVER(PARTITION BY fct.CustomerID) AS Total_Orders,
  SUM(Quantity) OVER(PARTITION BY fct.CustomerID) AS Total_Items,
  SUM(TotalPrice) OVER(PARTITION BY fct.CustomerID) AS Total_Purchase
FROM `finalproject-g2df12.retail.fct_invoices` AS fct
LEFT JOIN `finalproject-g2df12.retail.dim_customers` AS cus
  ON fct.CustomerID = cus.CustomerID
LEFT JOIN `finalproject-g2df12.retail.rfm_segmentation` AS rfm
  ON fct.CustomerID = rfm.CustomerID
WHERE fct.CustomerID IS NOT NULL
ORDER BY CustomerID

In [ ]:
CREATE OR REPLACE TABLE `finalproject-g2df12.retail.report_transactions` AS

SELECT
  InvoiceNo,
  DATE(InvoiceDate) AS Transaction_Date,
  CustomerID,
  SUM(Quantity) AS Total_Items,
  SUM(TotalPrice) AS Total_Sales
FROM `finalproject-g2df12.retail.fct_invoices`
WHERE CustomerID IS NOT NULL
GROUP BY CustomerID, InvoiceNo, InvoiceDate